In [4]:
#IMPORTAMOS LAS LIBRERIAS QUE SE USARAN EN EL ETL
import numpy as np
import pandas as pd
import json
import ast

In [5]:
#cargamos el dataset .csv 
movies_dataset=pd.read_csv('datasets\movies_dataset.csv')

C:\Users\CARRILLO JARA\AppData\Local\Temp\ipykernel_5084\4262558520.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_dataset=pd.read_csv('datasets\movies_dataset.csv')


In [6]:
movies_dataset.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


*Transformaciones

In [7]:
#función para desanidar la columna belongs_to_collection
def desanidar1(obj): 
    if isinstance(obj, str) and '{' in obj:
        dic = ast.literal_eval(obj)
        return dic['name']

In [8]:
#aplica la funcion al dataset
movies_dataset['belongs_to_collection']=movies_dataset['belongs_to_collection'].apply(desanidar1)

In [9]:
# #función para desanidar las columnas genres, production_companies, spoken_languages, production_countries
def desanidar2(obj): 
    if isinstance(obj, str) and '{' in obj:
        L = []
        for i in ast.literal_eval(obj):
            L.append(i['name'])
        return L

In [10]:
#aplica la funcion al df_anidados
movies_dataset['genres']=movies_dataset['genres'].apply(desanidar2)
movies_dataset['production_companies']=movies_dataset['production_companies'].apply(desanidar2)
movies_dataset['spoken_languages']=movies_dataset['spoken_languages'].apply(desanidar2)
movies_dataset['production_countries']=movies_dataset['production_countries'].apply(desanidar2)

In [11]:
movies_dataset.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,Toy Story Collection,30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,[English],Released,NaN,Toy Story,False,7.7,5415.0
1,False,None,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[English, Français]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,Grumpy Old Men Collection,0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,[English],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [12]:
#visulizar todas las columnas sin recortar
pd.set_option('display.max_columns', None)

In [13]:
#revisamos los campos nulos
movies_dataset.isnull().sum()

adult                        0
belongs_to_collection    40975
budget                       0
genres                    2442
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies     11881
production_countries      6288
release_date                87
revenue                      6
runtime                    263
spoken_languages          3835
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [14]:
#Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0
movies_dataset['revenue'].fillna(0, inplace=True)
movies_dataset['budget'].fillna(0, inplace=True)

In [15]:
movies_dataset.isnull().sum()

adult                        0
belongs_to_collection    40975
budget                       0
genres                    2442
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies     11881
production_countries      6288
release_date                87
revenue                      0
runtime                    263
spoken_languages          3835
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [16]:
#Los valores nulos del campo release_date deben eliminarse.
movies_dataset.dropna(subset=['release_date'], inplace=True)

In [17]:
#De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

#ver el tipo de dato que contiene la columna
print(movies_dataset['release_date'].dtypes)

object


In [18]:
#convertir los datos a datatime
#movies_dataset['release_date'] = pd.to_datetime(movies_dataset['release_date'], errors='coerce')
movies_dataset['release_date'] = pd.to_datetime(movies_dataset['release_date'], format='%Y-%m-%d', errors='coerce')

In [19]:
#vemos si quedan valores nulos despues de la transformación
movies_dataset.isnull().sum()

adult                        0
belongs_to_collection    40891
budget                       0
genres                    2384
homepage                 37610
id                           0
imdb_id                     14
original_language           11
original_title               0
overview                   941
popularity                   2
poster_path                339
production_companies     11799
production_countries      6214
release_date                 3
revenue                      0
runtime                    249
spoken_languages          3771
status                      83
tagline                  24981
title                        3
video                        3
vote_average                 3
vote_count                   3
dtype: int64

In [20]:
#volvemos a eliminar los nulos
movies_dataset.dropna(subset=['release_date'], inplace=True)

In [21]:
#crear año de estreno a partir de la columna release_date
movies_dataset['release_year'] = movies_dataset['release_date'].dt.year

In [22]:
#ver el tipo de dato que contiene la columna
print(movies_dataset['release_date'].dtypes)

datetime64[ns]


In [23]:
#otro metodo para el tipo de dato objeto
#movies_dataset['release_year'] = movies_dataset['release_date'].str.split('-').str[0]

In [24]:
movies_dataset.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,release_year
0,False,Toy Story Collection,30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,81.0,[English],Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,None,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,262797249.0,104.0,"[English, Français]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,Grumpy Old Men Collection,0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,0.0,101.0,[English],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995


In [25]:
movies_dataset.isnull().sum()

adult                        0
belongs_to_collection    40888
budget                       0
genres                    2384
homepage                 37610
id                           0
imdb_id                     14
original_language           11
original_title               0
overview                   941
popularity                   0
poster_path                339
production_companies     11796
production_countries      6211
release_date                 0
revenue                      0
runtime                    246
spoken_languages          3768
status                      80
tagline                  24978
title                        0
video                        0
vote_average                 0
vote_count                   0
release_year                 0
dtype: int64

In [26]:
#ver el tipo de dato que contiene la columna
print(movies_dataset['release_year'].dtypes)

int32


In [27]:
#cambiamos el tipo de dato de objet a float
movies_dataset['revenue'] = movies_dataset['revenue'].astype(float)
movies_dataset['budget'] = movies_dataset['budget'].astype(float)

In [28]:
#Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, 
#cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

movies_dataset['return'] = movies_dataset.apply(lambda x: (x['revenue'] / x['budget']) if (x['budget'] != 0) else 0, axis=1)

In [29]:
movies_dataset.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,release_year,return
0,False,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,81.0,[English],Released,NaN,Toy Story,False,7.7,5415.0,1995,12.451801
1,False,None,65000000.0,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,262797249.0,104.0,"[English, Français]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,4.043035
2,False,Grumpy Old Men Collection,0.0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,0.0,101.0,[English],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,0.000000


In [30]:
#Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,vote_count,poster_path y homepage.
movies_dataset.drop(['video', 'imdb_id', 'original_title', 'adult','vote_count','poster_path','homepage'], axis=1, inplace=True)

In [31]:
movies_dataset.head(3)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,release_year,return
0,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,81.0,[English],Released,NaN,Toy Story,7.7,1995,12.451801
1,None,65000000.0,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,262797249.0,104.0,"[English, Français]",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,1995,4.043035
2,Grumpy Old Men Collection,0.0,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,0.0,101.0,[English],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,1995,0.000000


In [32]:
#exportamos el data set limpio
#movies_dataset.to_csv('movies_clean.csv', index=False)

*CONSTRUCCIÓN DE LAS FUNCIONES

In [33]:
#funcion de consulta de películas por mes
def peliculas_mes(mes):

    mes = mes.lower()
    meses = {'enero': 1,'febrero': 2,'marzo': 3,'abril': 4,'mayo': 5,'junio': 6,'julio': 7,'agosto': 8,'septiembre': 9,'octubre': 10,'noviembre': 11,'diciembre': 12}
    mes_numero = meses[mes]

    # Convertir la columna "fecha" a un objeto de tipo fecha
    movies_dataset['release_date'] = pd.to_datetime(movies_dataset['release_date'])

    try:
        month_filtered = movies_dataset[movies_dataset['release_date'].dt.month == mes_numero]
    except (ValueError, KeyError, TypeError):
        return None

    month_unique = month_filtered.drop_duplicates(subset='id')
    respuesta = month_unique.shape[0]

    return {'mes':mes, 'cantidad':respuesta}

In [34]:
peliculas_mes('enero')

{'mes': 'enero', 'cantidad': 5909}